In [1]:
def install_dependencies():
    import os

    commands = [

        "pip install pinecone-client",
        "pip install chromadb langchain tiktoken",
        "pip install Chroma",
        "pip install -U langchain-community",
        "pip install langchain_together",
        "pip install sentence-transformers",
        "pip install boto3 requests",
        "pip install transformers datasets faiss-cpu",
        "pip install python-docx pandas",
        "pip install qdrant-client",
        "pip install chromadb",
        "pip install pymilvus",
        "pip install weaviate-client",
        "pip install langchain-weaviate",
        "pip install pinecone[grpc]",
        "pip install -U langchain-huggingface"

    ]

    for command in commands:
        os.system(command)

# Call the function
install_dependencies()


In [2]:
def preprocess_text(word_file_path, size):

  import os


  # Step 2: Read the Word file if it exists
  from docx import Document

  # Path to the Word file in the Kaggle dataset


  # Read the Word file
  doc = Document(word_file_path)

  # Extract text from the document
  full_text = []
  for para in doc.paragraphs:
      full_text.append(para.text)

  # Join all paragraphs into a single string
  document_text = '\n'.join(full_text)
  #print(document_text)
  from langchain.text_splitter import RecursiveCharacterTextSplitter
  from langchain.docstore.document import Document

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=200)
  # Convert the string to a Document object
  docs = [Document(page_content=document_text)]
  text = text_splitter.split_documents(docs) # Pass the Document object to split_documents
  #/kaggle/input/dataset1-fd/FD.docx
  return text



In [3]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_together import ChatTogether

def chroma_pipeline(word_file_path, embedding_model_name, size, chat_model, question, persist_directory="db"):

    # Step 1: Preprocess the text
    text = preprocess_text(word_file_path, size)

    # Step 2: Initialize the embedding model
    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    # Step 3: Create Chroma vector database and retriever
    vectordb = Chroma.from_documents(
        documents=text,
        embedding=embedding_model,
        persist_directory=persist_directory
    )
    vectordb.persist()
    retriever = vectordb.as_retriever()

    # Step 4: Initialize chat model
    chat_model_instance = ChatTogether(
        together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
        model=chat_model,
    )

    # Step 5: Define prompt template
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=(
            "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
            "Context: {context}\n\n"
            "Question: {question}\n\n"
            "Answer (be specific and avoid hallucinations):"
        )
    )

    # Step 6: Perform RetrievalQA
    qa_chain = RetrievalQA.from_chain_type(
        llm=chat_model_instance,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template},
    )

    llm_response = qa_chain.invoke(question)
    return llm_response['result']


In [4]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain_together import ChatTogether
import faiss
import numpy as np


def faiss_pipeline(word_file_path, embedding_model_name, size, chat_model, question, k=1):

    # Step 1: Preprocess the text
    text = preprocess_text(word_file_path, size)

    # Step 2: Initialize embedding model
    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)
    texts = [doc.page_content for doc in text]
    embeddings = embedding_model.embed_documents(texts)
    embeddings = np.array(embeddings)
    dimension = embeddings.shape[1]

    # Step 3: Create FAISS index
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
    index_to_docstore_id = {i: i for i in range(len(texts))}

    vector_store = FAISS(
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
        embedding_function=embedding_model.embed_query
    )

    # Step 4: Initialize chat model
    chat_model_instance = ChatTogether(
        together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
        model=chat_model,
    )

    # Step 5: Define prompt template
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=(
            "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
            "Context: {context}\n\n"
            "Question: {question}\n\n"
            "Answer (be specific and avoid hallucinations):"
        )
    )

    qa_chain = LLMChain(
        llm=chat_model_instance,
        prompt=prompt_template
    )

    query_embedding = embedding_model.embed_query(question)
    D, I = index.search(np.array([query_embedding]), k=1)

    doc_id = I[0][0]
    document = docstore.search(doc_id)
    context = document.page_content

    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)
    print(answer)


In [5]:
from qdrant_client import QdrantClient, models
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_together import ChatTogether
import numpy as np

def qdrant_pipeline(word_file_path, embedding_model_name, size, chat_model, question, batch_size=250, limit=2):

    # Preprocess the text from the provided file path
    text = preprocess_text(word_file_path, size)

    # Initialize embedding model
    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    # Generate embeddings for the text
    texts = [doc.page_content for doc in text]
    embeddings = embedding_model.embed_documents(texts)
    embeddings = np.array(embeddings)

    # Hard-coded Qdrant details
    client_url = "https://186e02e2-6d10-4b48-baf1-273a91f6c628.us-east4-0.gcp.cloud.qdrant.io:6333"
    client_api_key = "khkhQd22_WZRUBXQg_kL_I08CH3L5HmuHGrbETbVaZlyzCQfyjG0_w"
    collection_name = "text_vectors"

    # Initialize Qdrant client
    client = QdrantClient(url=client_url, api_key=client_api_key)

    # Recreate the collection in Qdrant
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=embeddings.shape[1], distance=models.Distance.COSINE)
    )

    # Upload the embeddings to Qdrant in batches
    qdrant_index = list(range(1, len(text) + 1))
    for i in range(0, len(text), batch_size):
        low_idx = min(i + batch_size, len(text))
        batch_of_ids = qdrant_index[i: low_idx]
        batch_of_embs = embeddings[i: low_idx]
        batch_of_payloads = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in text[i: low_idx]]

        client.upsert(
            collection_name=collection_name,
            points=models.Batch(
                ids=batch_of_ids,
                vectors=batch_of_embs.tolist(),
                payloads=batch_of_payloads
            )
        )

    # Query the Qdrant collection using the provided question
    query_embedding = embedding_model.embed_query(question)
    query_embedding = np.array(query_embedding)

    # Perform search on Qdrant to find the top-k most similar documents
    search_results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=limit
    )

    # Collect the context (page content) of the retrieved documents
    contexts = [result.payload['page_content'] for result in search_results]
    context = "\n".join(contexts)

    # Prepare the prompt for the Chat model
    prompt = f"""
    You are a helpful assistant. Use the following retrieved documents to answer the question:
    Context:
    {context}
    Question: {question}
    Answer:
    """

    # Perform inference using the Chat model
    llm = ChatTogether(api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9", model=chat_model)
    response = llm.predict(prompt)

    # Return the response
    return response


In [6]:
!pip install weaviate-client

  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymilvus 2.5.3 requires grpcio<=1.67.1,>=1.49.1, but you have grpcio 1.69.0 which is incompatible.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.


In [7]:
import numpy as np
import os
from langchain.embeddings import SentenceTransformerEmbeddings
import weaviate
from weaviate.auth import AuthApiKey
import weaviate.classes.config as wvcc
from weaviate.collections import Collection
from weaviate.auth import AuthApiKey
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_together import ChatTogether
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def weaviate_pipeline(word_file_path, embedding_model_name, chat_model, question):
    # Step 1: Preprocess the text and embed it using the embedding model
    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    # Set up Weaviate environment
    os.environ["WEAVIATE_URL"] = "https://9f6vknbcqwgzh4q4j8srna.c0.asia-southeast1.gcp.weaviate.cloud"
    os.environ["WEAVIATE_API_KEY"] = "CM0FKsSkL5gndIvZdYeuSc6vNY8f3MdApgGt"

    # Initialize Weaviate client
    weaviate_url = os.environ["WEAVIATE_URL"]
    weaviate_api_key = os.environ["WEAVIATE_API_KEY"]
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=AuthApiKey(weaviate_api_key),
    )

    # Preprocess the documents and create the Weaviate VectorStore
    text = preprocess_text(word_file_path, size=1000)  # Adjust the size parameter accordingly
    vs = WeaviateVectorStore.from_documents(
        documents=text,
        embedding=embedding_model,
        client=client
    )

    # Step 2: Inference with the chat model and retrieved context from Weaviate
    chat_model = ChatTogether(
        together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
        model=chat_model,
    )

    # Define the prompt template
    template = """You are an expert financial advisor. Use the context to answer questions accurately and concisely:
    Context:
    {context}
    Question: {question}
    Answer (be specific and avoid hallucinations):
    """

    prompt = ChatPromptTemplate.from_template(template)

    # Set up the retriever and RAG (Retrieval-Augmented Generation) chain
    retriever = vs.as_retriever()
    output_parser = StrOutputParser()
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | chat_model
        | output_parser
    )

    # Run the chain to get the result
    result = rag_chain.invoke(question)

    return result


In [8]:
def inference(vectordb_name, word_file_path, embedding_model_name, chat_model, question, size=1000, batch_size=250, top_k=2, limit=2):
    if vectordb_name == "Chroma":
        answer=chroma_pipeline(word_file_path, embedding_model_name, size, chat_model, question, persist_directory="db")
        print(answer)
    elif vectordb_name == "FAISS":
        answer=faiss_pipeline(word_file_path, embedding_model_name, size, chat_model, question, k=1)
        print(answer)
    elif vectordb_name == "Qdrant":
        answer=qdrant_pipeline(word_file_path, embedding_model_name, size, chat_model, question, batch_size=batch_size, limit=limit)
        print(answer)
    elif vectordb_name == "Weaviate":
        answer=weaviate_pipeline(word_file_path, embedding_model_name, chat_model, question)
        print(answer)
    else:
        print("Invalid Choice")


In [11]:
word_file_path="/content/FD.docx"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
vectordb_name = "Chroma"  # "Chroma" or "FAISS" or "Qdrant" or "Pinecone" or "Weaviate"
chat_model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"  # Qwen/QwQ-32B-Preview , meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo, meta-llama/Llama-3.3-70B-Instruct-Turbo
question = "What is the minimum amount for investing in FD?"

# Call the inference function
# inference(vectordb_name,chat_model,question)
inference(vectordb_name, word_file_path,embedding_model_name,chat_model, question)

<ipython-input-3-17b8e056e3ac>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-3-17b8e056e3ac>:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


The minimum amount for investing in Bajaj Finance Fixed Deposit is Rs. 15,000.
